In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [3]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [6]:
cat = Table.read('./Catalog/COSMOS/lsbg_COSMOS_after_cuts_JG.fits')

In [7]:
len(cat)

77

In [10]:
category = "LSBG"

### Generate RGB images

In [11]:
from kuaizi.display import display_HSC_cutout_rgb

In [12]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/COSMOS/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/COSMOS/Cutout_RGB/'
WEBPAGE_FIG_DIR = f'/tigress/jiaxuanl/public_html/COSMOS/cutout_RGB/z002_004/figure/{category}/'

In [14]:
problem_ids = []
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"cosmos_{obj['viz-id']}_{filt}.fits"):
            #print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")
            problem_ids.append(obj['viz-id'])
            
if not os.path.isdir(FIGURE_DIR):
    os.makedirs(FIGURE_DIR)

In [15]:
print('# of no cutouts:', np.unique(problem_ids).shape)

# of no cutouts: (0,)


In [16]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/cosmos_{ind}_{filt}.fits')[1].data for filt in 'griz'])
        fig, ax = plt.subplots(figsize=(5, 5))
        size = int(obj['rhalf_ellip'] * 5) if obj['rhalf_ellip'] * 5 > 100 else 100
        ax = display_HSC_cutout_rgb(images, ax=ax, half_width=size)
        plt.savefig(FIGURE_DIR + f'cosmos_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 77
Progress: 10 / 77
Progress: 20 / 77
Progress: 30 / 77
Progress: 40 / 77
Progress: 50 / 77
Progress: 60 / 77
Progress: 70 / 77


In [18]:
# Copy figures in FIGURE_DIR to WEBPAGE_FIG_DIR
if not os.path.isdir(WEBPAGE_FIG_DIR):
    os.makedirs(WEBPAGE_FIG_DIR)
else:
    import glob
    fileList = glob.glob(WEBPAGE_FIG_DIR + '*.png', recursive=True)
    [os.remove(file) for file in fileList];
            
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try: 
        copyfile(FIGURE_DIR + f'cosmos_{ind}_cutout.png', WEBPAGE_FIG_DIR + f'cosmos_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

### Generate webpage for cutouts. We use `diezi/webpage/gen_web_cutout.py`

In [20]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [21]:
#FOR NSA SAMPLE
WEBPAGE_FIG_DIR = f'/tigress/jiaxuanl/public_html/COSMOS/cutout_RGB/figure/{category}/'
WEBPAGE_DIR = f'/tigress/jiaxuanl/public_html/COSMOS/cutout_RGB/page/{category}/'
SCARLET_DIR = None #'/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'COSMOS'

In [23]:
webpage_cutout(WEBPAGE_FIG_DIR, WEBPAGE_DIR, sample_name, sample_title=f'{category}', SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 77 galaxies to be displayed
Total pages: 4


In [24]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/COSMOS/cutout_RGB/page/LSBG/


### The search box on webpage still doesn't work. Need to work on the javascript file